In [1]:
import pandas as pd 
import os
import numpy as np
import json 
import torch
from sklearn.preprocessing import StandardScaler


### Predicting Causal Estimates with Limited Supervision
In this notebook we:

- Utilize a graph neural network to predict the effect of a marketing campaign to the user's consumption.

- Evaluate the impact of supervision i.e. how many users have already participated in the campaign and what was the observed outcome.

- We compare our model with standard causal machine learning methods.

- We evaluate the method in terms of both uplift and accuracy metrics.


### Load the data

In [2]:
import torch

data = torch.load("/Users/georgepanagopoulos/Desktop/research/causal_inference/data/retailhero/processed/data.pt")[0]
data

HeteroData(
  (user, buys, product)={
    edge_index=[2, 14543339],
    treatment=[14543339],
  },
  user={
    x=[180653, 7],
    t=[180653],
    y=[180653],
  },
  products={ num_products=40542 }
)

### Model setup

In [3]:
from torch import nn
from torch_geometric.nn import SAGEConv

class BipartiteSAGE2mod(torch.nn.Module):
    def __init__(self, nfeat:int, nproduct:int , hidden_channels:int , out_channels: int, num_layers:int, dropout_rate:float =0):
        super().__init__()
        self.convs = torch.nn.ModuleList()

        self.user_embed = nn.Linear(nfeat, hidden_channels )
        self.item_embed =  nn.Linear(nproduct, hidden_channels)

        self.convs = torch.nn.ModuleList()

        for _ in range(num_layers):
            self.convs.append(SAGEConv((-1,-1), hidden_channels))
            
        #self.lin_hetero = Linear(hidden_channels, out_channels)
        
        self.num_layers = num_layers

        self.hidden_common1 = nn.Linear(hidden_channels + num_layers*hidden_channels, hidden_channels)
        self.hidden_common2 = nn.Linear(hidden_channels, hidden_channels)

        self.hidden_control = nn.Linear(hidden_channels, int(hidden_channels/2))
        self.hidden_treatment = nn.Linear(hidden_channels, int(hidden_channels/2))

        self.out_control = nn.Linear( int(hidden_channels/2), out_channels)
        self.out_treatment = nn.Linear( int(hidden_channels/2), out_channels)

        #self.lin = Linear(hidden_channels, out_channels)
        self.dropout = nn.Dropout(dropout_rate)
        #self.bn_hidden = nn.BatchNorm1d(hidden_channels)
        
        #self.bn_out = nn.BatchNorm1d(nfeat + hidden_channels + hidden_channels)
        self.activation = nn.ReLU()


    def forward(self, xu: torch.tensor, xp:torch.tensor, edge_index:torch._tensor):
        out = [] 
        xu = self.user_embed(xu)
        xp = self.item_embed(xp)

        out.append(xu)

        embeddings = torch.cat((xu,xp), dim=0) 
        
        for i in range(self.num_layers):
            embeddings = self.activation(self.convs[i](embeddings, edge_index))
            #embeddings = self.dropout(embeddings)
            #embeddings = self.bn_hidden(embeddings)
            
            out.append(embeddings[:xu.shape[0]])            
        
        out = torch.cat( out, dim=1)
        
        hidden = self.dropout(self.activation(self.hidden_common1(out)))
        hidden = self.dropout(self.activation(self.hidden_common2(hidden)))
        
        # separate treatment and control 
        hidden_1t0 = self.dropout(self.activation(self.hidden_control(hidden)))
        hidden_1t1 = self.dropout(self.activation(self.hidden_treatment(hidden)))

        out_2t0 = self.activation(self.out_control(hidden_1t0))
        out_2t1 = self.activation(self.out_treatment(hidden_1t1))
        
        
        return out_2t1, out_2t0, hidden_1t1, hidden_1t0

### Loss and Evaluation Functions

In [6]:
import torch.nn.functional as F

def outcome_regression_loss(t_true: torch.tensor,y_treatment_pred: torch.tensor, y_control_pred: torch.tensor, y_true: torch.tensor) -> torch.tensor:
    """
    Compute mse for treatment and control output layers using treatment vector for masking out the counterfactual predictions
    """
    loss0 = torch.mean((1. - t_true) * F.mse_loss(y_control_pred.squeeze(), y_true, reduction='none')) 
    loss1 = torch.mean(t_true *  F.mse_loss(y_treatment_pred.squeeze(), y_true, reduction='none') )

    return loss0 + loss1

In [7]:
def uplift_score(prediction, treatment, target, rate=0.2) -> float:
    """
    From https://ods.ai/competitions/x5-retailhero-uplift-modeling/data
    Order the samples by the predicted uplift. 
    Calculate the average ground truth outcome of the top rate*100% of the treated and the control samples.
    Subtract the above to get the uplift. 
    """
    order = np.argsort(-prediction)
    treatment_n = int((treatment == 1).sum() * rate)
    treatment_p = target[order][treatment[order] == 1][:treatment_n].mean()

    control_n = int((treatment == 0).sum() * rate)
    control_p = target[order][treatment[order] == 0][:control_n].mean()
    score = treatment_p - control_p
    return score

### Define training parameters

In [8]:
with open('/Users/georgepanagopoulos/Desktop/research/causal_inference/code/UMGNet/src/config_RetailHero.json', 'r') as config_file:
        config = json.load(config_file)
        
n_hidden = config["n_hidden"]
no_layers = config["no_layers"]
out_channels = config["out_channels"]
num_epochs = config["num_epochs"]
lr = config["lr"]
results_file_name = config['results_file_name']
model_file_name = config["model_file"]
early_thres = config['early_stopping_threshold']
l2_reg = config['l2_reg']
with_lp = config['with_label_prop'] == 1
number_of_runs = config['number_of_runs']
dropout = config['dropout']
k = 10
seed = 1
validation_fraction = 5
patience = 50

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

xp = torch.eye(data['products']['num_products']).to(device)
xu = data['user']['x'].to(device)
treatment = data['user']['t'].to(device)
outcome = data['user']['y'].to(device)

edge_index = data['user','buys','product']

criterion = outcome_regression_loss

In [12]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=abs(k), shuffle=True, random_state=seed)
result_fold = []
if with_lp:   
    dummy_product_labels = torch.zeros([xp.shape[0],1]).to(device).squeeze()

for train_indices, test_indices in kf.split(xu):
    test_indices, train_indices = train_indices, test_indices
    break 

# split the test indices to test and validation 
val_indices = train_indices[:int(len(train_indices)/validation_fraction)]
train_indices = train_indices[int(len(train_indices)/validation_fraction):]

### Subset the graph for treated edges and training users

In [14]:
## Keep the graph before the treatment and ONLY the edges of the the train nodes (i.e. after the treatment)
mask = torch.isin(data['user','buys','product']['edge_index'][0, :], torch.tensor(train_indices) )
edge_index_up_current = data['user','buys','product']['edge_index'][ : , (~data['user','buys','product']['treatment']) | (mask) ]

edge_index_up_current[1] = edge_index_up_current[1]+ data['user']['x'].shape[0]

edge_index_up_current = torch.cat([edge_index_up_current,edge_index_up_current.flip(dims=[0])],dim=1).to(device)

### Training

In [18]:
from typing import Callable
import torch_geometric as pyg
from torch.optim import Optimizer


def train(mask: torch.tensor, model:torch.nn.Module, xu: torch.tensor, xp: torch.tensor, edge_index: torch.tensor, treatment: torch.tensor, outcome: torch.tensor,
                  optimizer: Optimizer, criterion: Callable[[torch.tensor, torch.tensor, torch.tensor, torch.tensor], torch.tensor] ):
      model.train()
      optimizer.zero_grad()  # Clear gradients.

      out_treatment, out_control, hidden_treatment, hidden_control = model(xu, xp, edge_index)
      loss = criterion(treatment[mask], out_treatment[mask], out_control[mask], outcome[mask])
     
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss


def test(mask: torch.tensor, model:torch.nn.Module, xu: torch.tensor, xp: torch.tensor, edge_index: torch.tensor, treatment: torch.tensor, outcome: torch.tensor,
                  criterion: Callable[[torch.tensor, torch.tensor, torch.tensor, torch.tensor], torch.tensor] ):
      model.eval()
      out_treatment, out_control, hidden_treatment, hidden_control = model(xu, xp, edge_index)
      loss = criterion(treatment[mask], out_treatment[mask], out_control[mask], outcome[mask])
      return loss


In [ ]:
from torch.optim import Adam

model = BipartiteSAGE2mod(xu.shape[1], xp.shape[1] , n_hidden, out_channels, no_layers, dropout).to(device)
optimizer = Adam(model.parameters(), lr=lr, weight_decay = l2_reg)

# init params
out = model( xu, xp , edge_index_up_current)

In [ ]:
model_file = ""
early_stopping = 0
train_losses = []
val_losses = []
best_val_loss = np.inf
print_per_epoch = 50

for epoch in range(num_epochs):
    train_loss = train(train_indices, model, xu, xp, edge_index_up_current, treatment, outcome, optimizer, criterion)
    val_loss = test(val_indices, model, xu, xp, edge_index_up_current, treatment, outcome, criterion)

    train_losses.append(float(train_loss.item())) 
    val_losses.append(float(val_loss.item()))

    if val_loss < best_val_loss:
        early_stopping=0
        best_val_acc = val_loss
        torch.save(model, model_file)
    else:
        early_stopping += 1
        if early_stopping > patience:
            print("early stopping..")
            break
            
    if epoch%print_per_epoch==0:
        print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f}, Tra: {train_loss:.4f}, Val: {val_loss:.4f}') #, Test: {test_acc:.4f}'


### Testing

In [ ]:
model = torch.load(model_file).to(device)
model.eval()

out_treatment, out_control, hidden_treatment, hidden_control = model(data['user']['xu'], xp, edge_index_up_current)

test_loss = criterion(data['user']['t'][mask], out_treatment[mask], out_control[mask], data['user']["y"][mask])

treatment_test = data['user']['t'][test_indices].detach().cpu().numpy()
outcome_test = data['user']['y'][test_indices].detach().cpu().numpy()
out_treatment = out_treatment.detach().cpu().numpy()
out_control = out_control.detach().cpu().numpy()


uplift = out_treatment[test_indices] - out_control[test_indices]
uplift = uplift.squeeze()

#mse = (uplift.mean() - (outcome_test[treatment_test==1].mean() - outcome_test[treatment_test==0].mean()))**2
print(f'mse {test_loss}')
up40 = uplift_score(uplift, treatment_test, outcome_test,0.4)
print(f'up40 {up40}')
up20 = uplift_score(uplift, treatment_test, outcome_test,0.2)
print(f'up20 {up20}')

### Benchmarks

In [ ]:
from causalml.inference.meta import BaseXClassifier, BaseSClassifier, BaseTClassifier,BaseRClassifier, BaseDRRegressor, BaseXRegressor, BaseSRegressor, BaseTRegressor, BaseRRegressor
from causalml.propensity import ElasticNetPropensityModel
from sklearn.linear_model import LogisticRegression
from causalml.inference.tree import UpliftTreeClassifier
from xgboost import XGBClassifier, XGBRegressor
from causalml.inference.tree.causal.causaltree import CausalTreeRegressor

train_indices = torch.cat((train_indices, val_indices), dim=0)
 

In [ ]:
learner = BaseTRegressor(learner = XGBRegressor())

learner.fit(X= xu[train_indices], y=outcome[train_indices], treatment= treatment[train_indices] )  
uplift=learner.predict(X = xu[train_indices], treatment = treatment[test_indices]).squeeze()

uplift=learner.predict(X = xu[test_indices], treatment= treatment[test_indices]).squeeze()

score40 = uplift_score(uplift, np.hstack([treatment[train_indices] ,treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.4)
score20 = uplift_score(uplift, np.hstack([treatment[train_indices],treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.2)
print(f'T-learner up40:{score40},{score20}')


In [ ]:
propensity_model = ElasticNetPropensityModel() #dic_mod[model_class]()

propensity_model.fit(X=xu[train_indices], y = treatment[train_indices])
p_train = propensity_model.predict(X=xu[train_indices])
p_test = propensity_model.predict(X=xu[test_indices])

learner = BaseXRegressor(learner = XGBRegressor())

learner.fit(X= xu[train_indices], y=outcome[train_indices], treatment= treatment[train_indices], p=p_train )  

uplift=learner.predict(X = xu[test_indices], treatment= treatment[test_indices]).squeeze()

score40 = uplift_score(uplift, np.hstack([treatment[train_indices] ,treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.4)
score20 = uplift_score(uplift, np.hstack([treatment[train_indices],treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.2)
print(f'X-learner up40:{score40},{score20}')

In [ ]:
learner = CausalTreeRegressor(control_name="0")
X_train = np.hstack(( treatment[train_indices].reshape(-1, 1), xu[train_indices]))
X_test = np.hstack((treatment[test_indices].reshape(-1, 1),  xu[test_indices]))
learner.fit( X = X_train, treatment = treatment[train_indices].astype(str), y=outcome[train_indices])
uplift = learner.predict( X = X_test).squeeze()

score40 = uplift_score(uplift, np.hstack([treatment[train_indices] ,treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.4)
score20 = uplift_score(uplift, np.hstack([treatment[train_indices],treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.2)
print(f'X-learner up40:{score40},{score20}')